In [ ]:
# parameters
datadir = ""
scan_numbers = ""
axes = ""
signal = ""

# Scan Processor

Load a generic scan and view the output

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import hdfmap
import nexus2srs

print(hdfmap.module_info())

In [ ]:
# Inputs
processing_directory = os.path.join(datadir, "processing")
if not os.path.isdir(datadir):
    raise ValueError("data directory not found")
if not os.path.isdir(processing_directory):
    raise ValueError("processing directory not found")

scan_numbers = np.fromstring(str(scan_numbers).strip('[]'), dtype=int, sep=',')
inpath = [os.path.join(datadir, f"{n}.nxs") for n in scan_numbers]
for file in inpath:
    if not os.path.exists(file):
        raise FileNotFoundError(f"file not found: {file}")

if len(scan_numbers) == 1:
    outpath = os.path.join(processing_directory, f"{scan_numbers[0]}.nxs")
else:
    outpath = os.path.join(processing_directory, f"{scan_numbers[0]}-{scan_numbers[-1]}.nxs")
print("inpath: ", inpath)
print("outpath: ", outpath)

In [ ]:
# HdfMap from NeXus file:
hmap = hdfmap.create_nexus_map(inpath[0])
if axes not in hmap:
    raise ValueError(f"axes {axes} not found")
if signal not in hmap:
    raise ValueError(f"signal {signal} not found")

with hmap.load_hdf(inpath) as nxs:
    axes_data = hmap.get_data(nxs, axes)
    signal_data = hmap.get_data(nxs, signal)
    title = hmap.format_hdf(nxs, "{entry_identifier}\n{scan_command}")
axes_label = hmap.get_path(axes)
signal_label = hmap.get_path(signal)

# plot the data (e.g. using matplotlib)
plt.figure()
plt.plot(axes_data, signal_data)
plt.xlabel(axes_label)
plt.ylabel(signal_label)
plt.title(title)

plt.savefig('/tmp/output.png')

In [ ]:
# Plot detector data
if 'IMAGE' in hmap:
    with hmap.load_hdf(inpath[0]) as nxs:
        index = np.argmax(signal_data)
        image = hmap.get_image(nxs, index)

    plt.figure()
    plt.imshow(image)
    plt.title(title + f'\n{axes_label}[{index}] = {axes_data[index]:.3}')
    plt.savefig('/tmp/image.png')
else:
    print('No image')


In [ ]:
nexus2srs.nxs2dat(inpath[0], '/tmp/nexus.txt')
